In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

100%|██████████| 46080/46080 [00:01<00:00, 38887.11it/s]


In [15]:
COLUMNS = [
    # "facts",
    "optimiser",
    "features",
    "iterations",
    "feature_pruning",
    "data_pruning",
]

def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.iterations == "4"]
    df = df[df.facts == "fd"]
    df = df[df.features == "wl"]
    df = df[df.data_pruning == "none"]
    # df = df[df.feature_pruning.isin({"none", "collapse-all"})]
    return df

for metric in [
    # "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    df = df.fillna(0)
    # display df without hiding rows
    with pd.option_context("display.max_rows", None):
        display(df)

solved


domain                                                       blocksworld  \
optimiser features iterations feature_pruning  data_pruning                
rank-svm  wl       2          collapse-all     none                   24   
                              collapse-layer   none                   24   
                              collapse-layer-x none                   16   
                              none             none                   23   
                   4          collapse-all     none                   23   
                              collapse-layer   none                   23   
                              collapse-layer-x none                   16   
                              none             none                   21   
                   6          collapse-all     none                   24   
                              collapse-layer   none                   23   
                              collapse-layer-x none                   15   
                              none             none                    0   
svr       wl       2          collapse-all     none                   12   
                              collapse-layer   none                   11   
                              collapse-layer-x none                   16   
                              none             none                   11   
                   4          collapse-all     none                   19   
                              collapse-layer   none                   10   
                              collapse-layer-x none                   16   
                              none             none                   21   
                   6          collapse-all     none                   18   
                              collapse-layer   none                   12   
                              collapse-layer-x none                   16   
                              none             none                   21   

domain                                                       childsnack  \
optimiser features iterations feature_pruning  data_pruning               
rank-svm  wl       2          collapse-all     none                   7   
                              collapse-layer   none                  11   
                              collapse-layer-x none                  11   
                              none             none                   7   
                   4          collapse-all     none                  12   
                              collapse-layer   none                  14   
                              collapse-layer-x none                  10   
                              none             none                   9   
                   6          collapse-all     none                  12   
                              collapse-layer   none                  11   
                              collapse-layer-x none                  11   
                              none             none                  11   
svr       wl       2          collapse-all     none                   6   
                              collapse-layer   none                   6   
                              collapse-layer-x none                   5   
                              none             none                   6   
                   4          collapse-all     none                   7   
                              collapse-layer   none                   6   
                              collapse-layer-x none                   5   
                              none             none                   7   
                   6          collapse-all     none                   4   
                              collapse-layer   none                   4   
                              collapse-layer-x none                   5   
                              none             none                   4   

domain                                                       ferry  mico

In [6]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"]
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            symbol="feature_pruning",
            line_dash="feature_pruning",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)